In [1]:
import pandas as pd
import numpy as np
import geemap 
import ee 
import os
import math 
ee.Initialize()
ee.Authenticate()

True

In [2]:

Map = geemap.Map()
rio = geemap.shp_to_ee(r"F:\夜间灯光\区域\全国.shp").geometry()
Map.addLayer(rio, {}, 'Boundary')



In [3]:
#DSMP传感器相互校正 
def DSMP(Y):
    start = ee.Date.fromYMD(Y, 1, 1)
    end = ee.Date.fromYMD(Y,12,31)
    dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")\
            .filterBounds(rio) \
            .filterDate(start, end)
    id = dmsp.aggregate_array('system:id').getInfo() #会返回一个包含该时间范围内所有影像的 ID 列表
    dmspA= ee.Image(id[0]).select('stable_lights') #id[0]对应所给csv数据 
    dmspB= ee.Image(id[1]).select('stable_lights')
    coef = pd.read_csv(r"E:\学习与作业\夜间灯光\处理\表\合并.csv")
    imgID = dmspA.id().getInfo()
    index = coef['wx'] ==imgID 
    c0,c1,c2,a,b, = coef.loc[index].iloc[0,1:6]
    correct = dmspA.expression("c0 + (c1 * X) + (c2 * X**2)",{
                'X': dmsp, 
                'c0': c0,   
                'c1': c1,   
                'c2': c2   
        })
    de = correct.where(correct.gt(63),63).where(correct.lt(6),0)
    # 饱和校正
    if len(id) == 2:
        cal_image = de.pow(b).multiply(a)  
        # 连续性校正
        intra = dmspA.where(dmspA.eq(0).Or(dmspB.eq(0)), 0)\
                        .where(dmspA.neq(0).And(dmspB.neq(0)),dmspA.add(dmspB).divide(2))
    elif len(id) ==1:
         intra = de 
    return intra
#for year in range(2000,2001):
#        H = DSMP(year+1)
#        inter = H.where(DSMP(year + 2).eq(0),0)\
#                    .where(DSMP(year + 2).gt(0).And(DSMP(year).gt(DSMP(year))),DSMP(year))
        
    
   
#geemap.ee_export_image_to_drive(inter.clip(rio),  description=str(year), fileNamePrefix='校正', folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
        

#c0,c1,c2是回归模型的参数，分别对应于模型中的截距、一次项系数和二次项系数
#DNk = a* DN * DN + b * DN + c 
#(c2,c1,c0)
#1 到 5 之间的像元可能是由于传感器噪声、背景亮度或其他环境因素产生的
# 这些值通常不会反映城市灯光等真实信号,所以选择以6为判断节点

           


In [ ]:
for year in range(2000,2001):
        H = DSMP(year+1)
        inter = H.where(DSMP(year + 2).eq(0),0)\
                    .where(DSMP(year + 2).gt(0).And(DSMP(year).gt(DSMP(year))),DSMP(year))
geemap.ee_export_image_to_drive(inter.clip(rio),  description=str(year), fileNamePrefix='校正', folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
    

NPP处理

In [4]:
def DSMPK(Y):
    start = ee.Date.fromYMD(Y, 1, 1)
    end = ee.Date.fromYMD(Y,12,31)
    dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")\
            .filterBounds(rio) \
            .filterDate(start, end)\
            .select('stable_lights')
    id = dmsp.aggregate_array('system:id').getInfo() #会返回一个包含该时间范围内所有影像的 ID 列表
    dmspA= ee.Image(id[0]).select('stable_lights') #id[0]对应所给csv数据 
    coef = pd.read_csv(r"E:\学习与作业\夜间灯光\处理\表\wx.csv")
    imgID = dmspA.id().getInfo()
    index = coef['wx'] ==imgID 
    a,b, = coef.loc[index].iloc[0,1:3]
    de = dmspA.where(dmspA.gt(63),63).where(dmspA.lt(6),0)
    if len(id) == 2:
        dmspB= ee.Image(id[1]).select('stable_lights')    
        # 饱和校正
        cal_image = de.pow(b).multiply(a)  
        # 连续性校正
        intra = cal_image.where(dmspA.eq(0).Or(dmspB.eq(0)), 0)\
                        .where(dmspA.neq(0).And(dmspB.neq(0)),dmspA.add(dmspB).divide(2))
    elif len(id) ==1:
         intra = de 
    return intra

In [9]:
year = 2013
inter_2013 = DSMPK(year)
#geemap.ee_export_image_to_drive(inter_2013.clip(rio),  description=str(year), fileNamePrefix='dsmp校正', folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
    

In [5]:
import os
start = ee.Date.fromYMD(2013, 1, 1)
end = ee.Date.fromYMD(2013,12,31)
dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")\
        .filterBounds(rio) \
        .filterDate(start, end)\
        .mean()
de2013k = dmsp.where(dmsp.gt(63),63).where(dmsp.lt(6),0)
#去除npp噪声
start = ee.Date.fromYMD(2013, 1, 1)
end = ee.Date.fromYMD(2013,12,31)
npp = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
    .filterBounds(rio) \
    .filterDate(start, end)\
    .select('avg_rad')\
    .mean()
K = npp.id().getInfo() 
npp = npp.where(npp.lt(0),0)   
dmsp_mask = de2013k.select(0).gt(0)  # 选择第一个波段,DMSP-OLS 中亮值大于 0 的区域
masked = npp.updateMask(dmsp_mask) #通过与 DMSP-OLS 数据对比，将 NPP-VIIRS 数据中所有DN 值为负的像元赋为 0 值
num  = masked.where(masked.gt(400), ee.Image(0)) #以 2013 年 DMSP-OLS 数据的亮值区域为掩膜，提取 2013 年的 NPP-VIIRS 图像
vule =num.focal_max(radius=1, units='pixels', iterations=5)
ed =masked.where(masked.gt(400),vule) 
zh = ed.where(ed.gt(63),63).where(ed.lt(6),0)
#geemap.ee_export_image_to_drive(zh.clip(rio),  description='zh', fileNamePrefix='zhnpp', folder="npp夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)



计算A B 

In [7]:
#NPP处理
A =34.63798 
B = 0.16337125
for y in range(2013,2016):
    start = ee.Date.fromYMD(y, 1, 1)
    end = ee.Date.fromYMD(y,12,31)
    npp = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
        .filterBounds(rio) \
        .filterDate(start, end)\
        .select('avg_rad')\
        .mean()
    #de = npp.where(npp.gt(63),63).where(npp.lt(6),0)
    over = npp.pow(B).multiply(A)
    geemap.ee_export_image_to_drive(over.clip(rio),  description=str(y), fileNamePrefix=str(y), folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
   

In [ ]:
#NPP处理
A =34.63798 
B = 0.16337125
for y in range(2013,2017):
    start = ee.Date.fromYMD(y, 1, 1)
    end = ee.Date.fromYMD(y,12,31)
    npp = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
        .filterBounds(rio) \
        .filterDate(start, end)\
        .select('avg_rad')\
        .mean()
#de = npp.where(npp.gt(63),63).where(npp.lt(6),0)
over = npp.pow(B).multiply(A)
geemap.ee_export_image_to_drive(over.clip(rio),  description='str(y)', fileNamePrefix='over', folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
   


In [ ]:
#DSMP饱和校正 
def BH(Y):
    start = ee.Date.fromYMD(Y, 1, 1)
    end = ee.Date.fromYMD(Y,12,31)
    dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")\
            .filterBounds(rio) \
            .filterDate(start, end)\
            .select('stable_lights')
    id = dmsp.aggregate_array('system:id').getInfo() #会返回一个包含该时间范围内所有影像的 ID 列表
    dmspA= ee.Image(id[0]).select('stable_lights') #id[0]对应所给csv数据 
    dmspB= ee.Image(id[1]).select('stable_lights')
    coef = pd.read_csv(r"E:\学习与作业\夜间灯光\处理\表\wx.csv")
    imgID = dmspA.id().getInfo()
    index = coef['wx'] ==imgID 
    a,b, = coef.loc[index].iloc[0,1:3]
    de = dmspA.where(dmspA.gt(63),63).where(dmspA.lt(6),0)
    cal = de.pow(b).multiply(a) 
    return cal

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [7]:

geemap.ee_export_image_to_drive(zh.clip(rio),  description='zh', fileNamePrefix='zhnpp', folder="npp夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
   

In [ ]:
#NPP数据处理
#两者进行连续性校正
#year = 2012
#H = DSMP(year+1)
#inter = H.where(DSMP(year + 2).eq(0),0)\
#                    .where(DSMP(year + 2).gt(0).And(DSMP(year).gt(DSMP(year))),DSMP(year))
qbh = BH(2013) 
zh_array = zh.toArray()
qbh_array = qbh.toArray()
# 将图像展平成一维向量
X = zh_array.reshape(-1, 1)
Y = qbh_array.reshape(-1, 1)
# 对 X 和 Y 进行对数变换
log_X = np.log(X)
log_Y = np.log(Y)
# 使用线性回归模型拟合
model = LinearRegression().fit(log_X, log_Y)
# 获取回归系数
A_log = model.intercept_  #ln(A)
B = model.coef_  # 斜率 B
A = np.exp(A_log)
A,B


radius=1 意味着邻域是一个 3x3 像素的方块，包含中心像元及其周围的 8 个像元（即1像素的半径）
units='pixels'单位是像素
iterations=5,5 次迭代能够确保某些边界的异常值被逐步替换








In [ ]:
#去除npp噪声
start = ee.Date.fromYMD(2012, 1, 1)
end = ee.Date.fromYMD(2012,12,31)
npp = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
    .filterBounds(rio) \
    .filterDate(start, end)\
    .select('avg_rad')\
    .mean()
K = npp.id().getInfo() 
npp = npp.where(npp.lt(0),0)   
dmsp_mask = de2013k.select(0).gt(0)  # 选择第一个波段,DMSP-OLS 中亮值大于 0 的区域
masked = npp.updateMask(dmsp_mask) #通过与 DMSP-OLS 数据对比，将 NPP-VIIRS 数据中所有DN 值为负的像元赋为 0 值
num  = masked.where(masked.gt(400), ee.Image(0)) #以 2013 年 DMSP-OLS 数据的亮值区域为掩膜，提取 2013 年的 NPP-VIIRS 图像
vule =num.focal_max(radius=1, units='pixels', iterations=5)
ed =masked.where(masked.gt(400),vule) 
zh = masked.where(masked.gt(63),63).where(dmsp.lt(6),0)
geemap.ee_export_image_to_drive(zh.clip(rio),  description='npp', fileNamePrefix='npp去除噪声', folder="夜间灯光", region=rio, scale=1000, crs='epsg:4326',maxPixels=1e12)
        

